# Question 3: Interactive Debugging Tool (bonus) 

In [6]:
import numpy as np
import pandas as pd

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('dataset/Allseasons.csv')

In [3]:
df

,eventType,period,periodTime,periodType,gameID,teamOfShooter,homeOrAway,xCoord,yCoord,shooter,goalie,shotType,emptyNet,strength,season,rinkSide,gameType,totalPlayTime
0,Shot,1,01:11,REGULAR,2016020001,Toronto Maple Leafs,away,-77.0,5.0,Mitchell Marner,Craig Anderson,Wrist Shot,NaN,NaN,2016,right,R,01:00:37
1,Shot,1,02:53,REGULAR,2016020001,Ottawa Senators,home,86.0,13.0,Chris Kelly,Frederik Andersen,Wrist Shot,NaN,NaN,2016,left,R,01:00:37
2,Shot,1,04:01,REGULAR,2016020001,Ottawa Senators,home,23.0,-38.0,Cody Ceci,Frederik Andersen,Wrist Shot,NaN,NaN,2016,left,R,01:00:37
3,Shot,1,04:46,REGULAR,2016020001,Ottawa Senators,home,33.0,-15.0,Erik Karlsson,Frederik Andersen,Slap Shot,NaN,NaN,2016,left,R,01:00:37
4,Shot,1,06:46,REGULAR,2016020001,Toronto Maple Leafs,away,-34.0,28.0,Martin Marincin,Craig Anderson,Wrist Shot,NaN,NaN,2016,right,R,01:00:37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385071,Shot,3,16:19,REGULAR,2020030415,Tampa Bay Lightning,home,57.0,30.0,Blake Coleman,Carey Price,Wrist Shot,NaN,NaN,2020,left,P,01:00:00
385072,Shot,3,17:13,REGULAR,2020030415,Tampa Bay Lightning,home,56.0,-31.0,Mikhail Sergachev,Carey Price,Backhand,NaN,NaN,2020,left,P,01:00:00
385073,Shot,3,19:01,REGULAR,2020030415,Montréal Canadiens,away,-45.0,-14.0,Tyler Toffoli,Andrei Vasilevskiy,Wrist Shot,NaN,NaN,2020,right,P,01:00:00
385074,Shot,3,19:10,REGULAR,2020030415,Montréal Canadiens,away,-77.0,-3.0,Cole Caufield,Andrei Vasilevskiy,Backhand,NaN,NaN,2020,right,P,01:00:00


# Process Data

In [4]:
def normalize_df(df):
    df_normalized = df.copy()
    df_normalized['xCoordNorm'] = np.where(df['rinkSide'] == 'left', df['xCoord'], -1 * df['xCoord'])
    df_normalized['yCoordNorm'] = np.where(df['rinkSide'] == 'left', df['yCoord'], -1 * df['yCoord'])

    return df_normalized

# Normalize Shot Direction
df_normalized = normalize_df(df)


# Grouped list of game IDs by season and game type
games = df.groupby(['season', 'gameType'])['gameID'].unique()

# Create Plot

In [7]:
def plot_one_event(event, normalize=False, title='', marker='o', size=100, color='#ea6969', alpha=.7):
    fig, ax = plt.subplots(figsize=(13,8.5))
    plt.axis('on')

    img = plt.imread('../figures/nhl_rink.png')
    # rink dimensions
    ext = [-100, 100, -42.5, 42.5]
    plt.imshow(img, zorder=0, extent=ext, aspect=1)

    # plot the points on top of the image
    xLabel, yLabel = ('xCoordNorm', 'yCoordNorm') if normalize else ('xCoord', 'yCoord')
    plt.scatter(event[xLabel], event[yLabel], s=size, c=color, alpha=alpha, marker=marker, label='shot')

    # plot the opposing team's net
    net_x, net_y = (89, 0) if (event['rinkSide'] == 'left') or normalize else (-89, 0)
    plt.scatter(net_x, net_y, s=200, c='blue', alpha=alpha, marker='x', label='net')
    plt.xlabel('feet')
    plt.ylabel('feet')
    plt.title(title)
    plt.legend()


@interact(
    season_year=widgets.IntSlider(min=2016, max=2020, step=1),
    season_type=['R', 'P']
)
def show_data(season_year, season_type):
    season_games = games[(season_year, season_type)]

    @interact(game_id=widgets.SelectionSlider(options=season_games))
    def show_game(game_id):
        selected_game = df_normalized[df_normalized['gameID'] == game_id]

        @interact(event_id=widgets.IntSlider(min=0, max=selected_game.shape[0]-1, step=1, value=0))
        def show_event(event_id, normalize_shots=True):
            event = selected_game.iloc[event_id]
            title = f"{event['eventType']} \t {event['shotType']} \t {event['shooter']} -> {event['goalie']}"
            plot_one_event(event, normalize=normalize_shots, title=title, marker='o', size=300, color='red', alpha=1)
            print(event)


interactive(children=(IntSlider(value=2016, description='season_year', max=2020, min=2016), Dropdown(descripti…